In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda

2024-01-31 18:47:07.317795: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 18:47:07.317857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 18:47:07.336559: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 18:47:07.505539: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 18:47:10.729108: W tensorflow/compiler/tf2

In [9]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

class CustomBiasLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomBiasLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Add a trainable weight for the bias
        self.bias = self.add_weight(
            shape=(input_shape[-1],),
            initializer='zeros',
            trainable=True,
            name='bias'
        )
        super(CustomBiasLayer, self).build(input_shape)

    def call(self, inputs):
        return inputs + self.bias


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda

# Re-define the model with the custom layer
model = Sequential([
    Lambda(lambda x: x * 2, input_shape=(10,)),  # Lambda layer
    CustomBiasLayer(),  # Custom bias layer
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary to see the updated model architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 10)                0         
                                                                 
 custom_bias_layer (CustomB  (None, 10)                10        
 iasLayer)                                                       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 21 (84.00 Byte)
Trainable params: 21 (84.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model("keras_lambda_model.h5")


In [14]:
from tensorflow.keras.models import load_model

# Assuming custom_layers.py contains your custom layer definitions
# from custom_layers import CustomLayer1, CustomLayer2

def count_layers_in_model(model_path):
    # Load the model
    model = load_model(model_path, custom_objects={
        # 'CustomLayer1': CustomLayer1,
        # 'CustomLayer2': CustomLayer2
    })
    
    # Initialize counters
    lambda_count = 0
    custom_count = 0
    
    # Iterate through the model's layers
    for layer in model.layers:
        if isinstance(layer, Lambda):
            lambda_count += 1
        # Uncomment and modify according to your custom layers
        # elif isinstance(layer, (CustomLayer1, CustomLayer2)):
        #     custom_count += 1
    
    return lambda_count, custom_count

# List of your .h5 model files
model_files = ['keras_lambda_model.h5']

# Iterate through the list and count layers for each model
for model_file in model_files:
    lambda_count, custom_count = count_layers_in_model(model_file)
    print(f'Model: {model_file}, Lambda Layers: {lambda_count}, Custom Layers: {custom_count}')


Model: keras_lambda_model.h5, Lambda Layers: 1, Custom Layers: 0


In [30]:
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
import tensorflow as tf

global standard_layer_names

def get_standard_keras_layer_names():
    layer_names = []
    # Iterate through attributes in `layers` module and collect layer names
    for attribute_name in dir(layers):
        attribute = getattr(layers, attribute_name)
        if issubclass(type(attribute), type(tf.keras.layers.Layer)):
            layer_names.append(attribute_name)
    return layer_names

def check_for_custom_layers(model_path):
    model = load_model(model_path)
    standard_layer_names = get_standard_keras_layer_names()
    custom_layers = []

    for layer in model.layers:
        print(layer.__class__.__name__)
        if layer.__class__.__name__ not in standard_layer_names:
            
            custom_layers.append(layer.__class__.__name__)
    
    return standard_layer_names, custom_layers

# Example usage
model_files = ['keras_lambda_model.h5']

for model_file in model_files:
    standard_layer_names, custom_layers = check_for_custom_layers(model_file)
    print(f"Model: {model_file} contains these non-standard layers: {custom_layers}")


Lambda
Dense
Model: keras_lambda_model.h5 contains these non-standard layers: []


In [28]:
print(standard_layer_names)

['AbstractRNNCell', 'Activation', 'ActivityRegularization', 'Add', 'AdditiveAttention', 'AlphaDropout', 'Attention', 'Average', 'AveragePooling1D', 'AveragePooling2D', 'AveragePooling3D', 'AvgPool1D', 'AvgPool2D', 'AvgPool3D', 'BatchNormalization', 'Bidirectional', 'CategoryEncoding', 'CenterCrop', 'Concatenate', 'Conv1D', 'Conv1DTranspose', 'Conv2D', 'Conv2DTranspose', 'Conv3D', 'Conv3DTranspose', 'ConvLSTM1D', 'ConvLSTM2D', 'ConvLSTM3D', 'Convolution1D', 'Convolution1DTranspose', 'Convolution2D', 'Convolution2DTranspose', 'Convolution3D', 'Convolution3DTranspose', 'Cropping1D', 'Cropping2D', 'Cropping3D', 'Dense', 'DenseFeatures', 'DepthwiseConv1D', 'DepthwiseConv2D', 'Discretization', 'Dot', 'Dropout', 'ELU', 'EinsumDense', 'Embedding', 'Flatten', 'GRU', 'GRUCell', 'GaussianDropout', 'GaussianNoise', 'GlobalAveragePooling1D', 'GlobalAveragePooling2D', 'GlobalAveragePooling3D', 'GlobalAvgPool1D', 'GlobalAvgPool2D', 'GlobalAvgPool3D', 'GlobalMaxPool1D', 'GlobalMaxPool2D', 'GlobalMaxPo